Segmentación del mercado de adolecentes 
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/predictive-modeling/blob/master/07-kmeans-R-teen-market-segments.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/predictive-modeling/blob/master/07-kmeans-R-teen-market-segments.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/predictive-modeling/blob/master/LICENCIA.txt)  
[Readme](https://github.com/jdvelasq/predictive-modeling/blob/master/readme.md)

# Definición del problema real

Un vendedor desea enviar publicidad electrónica a una población de adolecentes y adultos jóvenes con el fin de maximizar sus ventas. Para ello, desea poder clasificar a sus clientes potenciales por grupos de interés de acuerdo con sus intereses y consecuentemente enviar publicidad específica a cada uno de ellos.   

# Definición del problema en términos de los datos

El problema consiste en poder determinar que grupos de interés existen en una población de clientes a partir de los mensajes enviados por un servicio de redes sociales. La información disponible consiste en 30000 observaciones de 40 variables que podrían caracterizar los intereses de la población analizada. Cada variable mide la frecuencia con que una determinada palabra aparece en los mensajes de texto; adicionalmente, dentro de estas variables se incluye  información como el sexo, la edad y la cantidad de contactos de la persona. 

# Exploración

In [1]:
# Carga de los datos
teens <- read.csv("data/snsdata.csv")
str(teens)

'data.frame':	30000 obs. of  40 variables:
 $ gradyear    : int  2006 2006 2006 2006 2006 2006 2006 2006 2006 2006 ...
 $ gender      : Factor w/ 2 levels "F","M": 2 1 2 1 NA 1 1 2 1 1 ...
 $ age         : num  19 18.8 18.3 18.9 19 ...
 $ friends     : int  7 0 69 0 10 142 72 17 52 39 ...
 $ basketball  : int  0 0 0 0 0 0 0 0 0 0 ...
 $ football    : int  0 1 1 0 0 0 0 0 0 0 ...
 $ soccer      : int  0 0 0 0 0 0 0 0 0 0 ...
 $ softball    : int  0 0 0 0 0 0 0 1 0 0 ...
 $ volleyball  : int  0 0 0 0 0 0 0 0 0 0 ...
 $ swimming    : int  0 0 0 0 0 0 0 0 0 0 ...
 $ cheerleading: int  0 0 0 0 0 0 0 0 0 0 ...
 $ baseball    : int  0 0 0 0 0 0 0 0 0 0 ...
 $ tennis      : int  0 0 0 0 0 0 0 0 0 0 ...
 $ sports      : int  0 0 0 0 0 0 0 0 0 0 ...
 $ cute        : int  0 1 0 1 0 0 0 0 0 1 ...
 $ sex         : int  0 0 0 0 1 1 0 2 0 0 ...
 $ sexy        : int  0 0 0 0 0 0 0 1 0 0 ...
 $ hot         : int  0 0 0 0 0 0 0 0 0 1 ...
 $ kissed      : int  0 0 0 0 5 0 0 0 0 0 ...
 $ dance       : int

In [2]:
table(teens$gender)


    F     M 
22054  5222 

In [1]:
# La función excluye los NA
# a continuación se tienen en cuenta en el conteo
table(teens$gender, useNA = "ifany")

ERROR: Error in table(teens$gender, useNA = "ifany"): object 'teens' not found


In [5]:
# La muestra contiene un rango de edades 
# por fuera de la población de interés
summary(teens$age)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  3.086  16.312  17.287  17.994  18.259 106.927    5086 

In [6]:
# Se seleccionan las personas entre 13 y 20 años.
teens$age <- ifelse(teens$age >= 13 & teens$age < 20, teens$age, NA)

In [8]:
# Se verifica la variable edad en los registros de las
# personas en la población de interés.
summary(teens$age)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  13.03   16.30   17.27   17.25   18.22   20.00    5523 

In [9]:
# Se crean nuevas variables numéricas a partir 
# de información categórica
teens$female <- ifelse(teens$gender == "F" & !is.na(teens$gender), 1, 0)
teens$no_gender <- ifelse(is.na(teens$gender), 1, 0)

In [10]:
# Cantidad de hombres y mujeres en la muestra.
table(teens$gender, useNA = "ifany")


    F     M  <NA> 
22054  5222  2724 

In [11]:
# Cantidad de hombres y mujeres en la muestra
# 1=Female
table(teens$female, useNA = "ifany")


    0     1 
 7946 22054 

In [2]:
# Cantidad de patrones para los que se 
# conoce y no se conoce el genero
table(teens$no_gender, useNA = "ifany")

ERROR: Error in table(teens$no_gender, useNA = "ifany"): object 'teens' not found


In [14]:
# La muestra contiene datos faltantes
mean(teens$age)

[1] NA

In [15]:
# Edad sin tener en cuenta los datos faltantes
mean(teens$age, na.rm = TRUE)

[1] 17.25243

In [16]:
# Edad por año de graduación
aggregate(data = teens, age ~ gradyear, mean, na.rm = TRUE)

gradyear,age
2006,18.65586
2007,17.70617
2008,16.76770
2009,15.81957


In [17]:
# Los valores faltantes se llenan con el promedio
# de acuerdo con el año de graduación
ave_age <- ave(teens$age, 
               teens$gradyear, 
               FUN = function(x) mean(x, na.rm = TRUE))
                   
teens$age <- ifelse(is.na(teens$age), ave_age, teens$age)
summary(teens$age)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  13.03   16.28   17.24   17.24   18.21   20.00 

# Metodología

Este es el problema inverso al de clasificación. Acá se desea determinar cuántos grupos hay en los datos. El algoritmo se basa en asignar $n$ ejemplos a uno de $k$ grupos posibles. Se pretende minimizar la distancia entre los miembros de cada grupo y maximizar la distancia entre grupos. El algoritmo funciona de la siguiente forma:

* $S_i$ representa el conjunto de patrones del grupo $i$.


* La pertenencia del ejemplo $x_j$ al grupo $S_i$ se representa como $x_j \in S_i$. 


* $u_i$ representa el centro del grupo $i$.


El algoritmo busca minimizar:

$$ \sum_{i=1}^k \sum_{x_j \in S_i} \| x_j - u_i \|^2$$

Este proceso se realiza en dos fases:


1. Dados los centros de los clusters $u_i$, cada punto $x_j$ se asigna al cluster más cercano. En esta fase se asignan todos los ejemplos de la muestra de datos a un cluster.


2. Se recalcula cada centro $u_i$ como el promedio de los puntos $x_j$ que pertenecen a él.


Los centros iniciales se pueden obtener por muestreo aleatorio de los datos.

El algoritmo se detiene cuando ningún punto cambia de cluster.

# Entrenamiento del modelo

In [18]:
# Se separa la información de las palabras relacionadas
# con los intereses del resto de la información
# y se escala para eliminar problemas asociados
# a la medida de los datos.
interests <- teens[5:40]
interests_z <- as.data.frame(lapply(interests, scale))

In [19]:
# Se usa el algoritmo para determinar los centros de 5 grupos
set.seed(2345)
teen_clusters <- kmeans(interests_z, 5)

# Evaluación del modelo

In [20]:
# Número de ejemplos en cada cluster
teen_clusters$size

[1]   871   600  5981  1034 21514

In [21]:
# Frecuencia de las palabras en cada cluster
teen_clusters$centers

,basketball,football,soccer,softball,volleyball,swimming,cheerleading,baseball,tennis,sports,⋯,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
1,0.16001227,0.2364174,0.10385512,0.07232021,0.18897158,0.23970234,0.3931445,0.02993479,0.13532387,0.10257837,⋯,0.06137340,0.60368108,0.79806891,0.5651537331,4.1521844,3.96493810,0.043475966,0.09857501,0.035614771,0.03443294
2,-0.09195886,0.0652625,-0.09932124,-0.01739428,-0.06219308,0.03339844,-0.1101103,-0.11487510,0.04062204,-0.09899231,⋯,-0.01146396,-0.08724304,-0.03865318,-0.0003526292,-0.1678300,-0.14129577,0.009447317,0.05135888,-0.086773220,-0.06878491
3,0.52755083,0.4873480,0.29778605,0.37178877,0.37986175,0.29628671,0.3303485,0.35231971,0.14057808,0.32967130,⋯,0.03471458,0.48318495,0.66327838,0.3759725120,-0.0553846,-0.07417839,0.037989066,0.11972190,-0.009688746,-0.05973769
4,0.34081039,0.3593965,0.12722250,0.16384661,0.11032200,0.26943332,0.1856664,0.27527088,0.10980958,0.79711920,⋯,0.36134138,0.62256686,0.27101815,1.2306917174,0.1610784,0.26324494,1.712181870,0.93631312,1.897388200,2.73326605
5,-0.16695523,-0.1641499,-0.09033520,-0.11367669,-0.11682181,-0.10595448,-0.1136077,-0.10918483,-0.05097057,-0.13135334,⋯,-0.02918252,-0.18625656,-0.22865236,-0.1865419798,-0.1557662,-0.14861104,-0.094875180,-0.08370729,-0.087520105,-0.11423381


**Ejercicio.--** Cuál de los clusters anteriores tiene más interés en los deportes?

**Ejercicio.--** Cuál es el cluster de las princesas?

**Ejercicio.--** Cuál cluster representa las personas que no hablan o no han posteado sobre sus intereses?

# Análisis del modelo

In [22]:
# Se asigna a cada ejemplo de los datos
# el cluster al que pertenece
teens$cluster <- teen_clusters$cluster

In [23]:
# clusters a los que pertenecen los primeros cinco patrones
teens[1:5, c("cluster", "gender", "age", "friends")]

cluster,gender,age,friends
5,M,18.982,7
3,F,18.801,0
5,M,18.335,69
5,F,18.875,0
4,NA,18.995,10


In [24]:
# Características demográficas de los clusters
aggregate(data = teens, age ~ cluster, mean)

cluster,age
1,16.86497
2,17.39037
3,17.07656
4,17.11957
5,17.29849


In [25]:
aggregate(data = teens, female ~ cluster, mean)

cluster,female
1,0.8381171
2,0.7250000
3,0.8378198
4,0.8027079
5,0.6994515


In [26]:
# Cantidad promedio de amigos por cluster
aggregate(data = teens, friends ~ cluster, mean)

cluster,friends
1,41.43054
2,32.57333
3,37.16185
4,30.50290
5,27.70052


**Ejercicio.--** Al analizar las distintas variables presentadas, qué puede inferir de cada cluster?

---

Segmentación del mercado de adolecentes 
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/predictive-modeling/blob/master/07-kmeans-R-teen-market-segments.ipynb) para acceder a la última versión online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/predictive-modeling/blob/master/07-kmeans-R-teen-market-segments.ipynb) para ver la última versión online en `nbviewer`. 

---
[Licencia](https://github.com/jdvelasq/predictive-modeling/blob/master/LICENCIA.txt)  
[Readme](https://github.com/jdvelasq/predictive-modeling/blob/master/readme.md)